In [1]:

import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [973 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# Show schema to confirm date type
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [5]:
# Import date time functions
from pyspark.sql.functions import year

# Show the year for the date column
# df.select(year(df["date"])).show()
df.select(year(df.date)).show()

+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [6]:
# Save the year as a new column
df = df.withColumn("year", year(df['date']))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
(SELECT Year, Round(avg(price),2) FROM home_sales
WHERE bedrooms ==4 
GROUP BY year)
"""
spark.sql(query).show()

+----+--------------------+
|Year|round(avg(price), 2)|
+----+--------------------+
|2022|           296363.88|
|2019|            300263.7|
|2020|           298353.78|
|2021|           301819.44|
+----+--------------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
(SELECT date_built, Round(avg(price),2) FROM home_sales
WHERE bedrooms ==3 AND  bathrooms==3
GROUP BY date_built)
"""
spark.sql(query).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [10]:
##  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#BY sqft_living
query = """
(SELECT date_built, Round(avg(price),2) FROM home_sales
WHERE bedrooms ==3 AND  bathrooms==3 AND floors==2 AND sqft_living>=2000
GROUP BY date_built)
"""
spark.sql(query).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           297609.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2012|           307539.97|
|      2016|            293965.1|
|      2010|           285010.22|
|      2011|           276553.81|
|      2017|           280317.58|
+----------+--------------------+



In [11]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
#BY sqft_lot
query = """
(SELECT date_built, Round(avg(price),2) FROM home_sales
WHERE bedrooms ==3 AND  bathrooms==3 AND floors==2 AND sqft_lot>=2000
GROUP BY date_built)
"""
spark.sql(query).show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           291788.36|
|      2013|           295142.13|
|      2014|           294195.13|
|      2012|           295858.68|
|      2016|            287812.6|
|      2010|           280447.23|
|      2011|           281413.45|
|      2017|           282026.59|
+----------+--------------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
(SELECT view, Round(avg(price),2) FROM home_sales
WHERE price>=350000
Group by view)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.5243918895721436 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [15]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query = """
(SELECT view, Round(avg(price),2) FROM home_sales
WHERE price>=350000
Group by view)
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.47350168228149414 seconds ---


In [16]:
df.rdd.isEmpty()

False

In [17]:
# Write out the data in parquet format
df.write.parquet('parquet_home_sales', mode='overwrite')

In [18]:
# Read in our new parquet formatted data
p_df=spark.read.parquet('parquet_home_sales')

In [19]:
# A parquet formatted DataFrame has all the same methods as a row-based dataframe
# We can convert the dataframe to a view.
p_df.createOrReplaceTempView('p_home_sales')

In [20]:
start_time = time.time()

query = """
(SELECT view, Round(avg(price),2) FROM p_home_sales
WHERE price>=350000
Group by view)
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.5289995670318604 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home_sales")

In [22]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet_home_sales')

In [23]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [24]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query = """
(SELECT view, Round(avg(price),2) FROM p_home_sales
WHERE price>=350000
Group by view)
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.5195162296295166 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False